## Basic agent with tools: text retrieval, visual retrieval, multimodal retrireval, sql, item-based collaborative filtering

In [1]:
SQL_PATH = '/Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movies_meta.db'
PLOT_PATH = "/Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movie_plots.csv"
REVIEW_PATH = '/Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/reviews_w_movies_full.csv'
POSTER_PATH = '/Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movie_posters.csv'

In [2]:
from src.langchain.chains.movie_rag import MovieRAGChain
from src. langchain.loaders import MoviePosterDocumentLoader
from src.retrievers.visual_retriever import VisualRetriever
from src.langchain.prompts import ZERO_SHOT_QA_PROMPT

In [3]:
poster_loader = MoviePosterDocumentLoader(POSTER_PATH, max_movies=1000)   # small number since model is local
visual_docs = poster_loader.load()
visual_retriever = VisualRetriever(model_name="ViT-B/32", use_text_fusion=True, alpha=0.8)
visual_retriever.add_documents(visual_docs)

MAX_MOVIES = 500  # Limit for faster demos

full_chain = MovieRAGChain(
    plots_path=PLOT_PATH,
    reviews_path=REVIEW_PATH,
    max_movies=MAX_MOVIES,
    use_custom_retriever=True,
    use_custom_chunk=True,
    custom_prompt=ZERO_SHOT_QA_PROMPT,
    k=5,
    use_hyde=True,
    hyde_model="gpt-4o-mini",
    use_reranking=True,
    reranker_cfg={'type':'llm'},
    initial_k=20, 
)
full_chain.build()

Loading posters from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movie_posters.csv...
Created 947 poster docs.
  ✓ 947 poster documents
Loading CLIP model: ViT-B/32
✓ CLIP loaded on cpu
✓ VisualRetriever (text_fusion=True, method=weight_average)
  Alpha: 0.8 (image=0.8, text=0.2)
Encoding 947 posters with CLIP...
Encoding 947 text descriptions with CLIP...
✓ Fused embeddings using weight_average method (dim=512)
✓ Added 947 posters to index
✓ MovieRAGChain initialized
  Retriever type: custom + reranking + HyDE
  LLM: gpt-4o-mini

Building RAG Pipeline

1. Loading documents...
Limiting to 500 movies
Loading plots from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movie_plots.csv...
Created 383 plot docs.
  ✓ 383 plot documents
Loading reviews from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/reviews_w_movies_full.csv...
Created 500 review docs.
  ✓ 500 review documents
✓ Total: 883 reviews and plots documents

2

In [3]:
# I wanna see what movies we have in dataset
#[chunk['metadata']['movie_title'] for chunk in full_chain.base_retriever.retriever.chunks if 'kung fu' in chunk['metadata']['movie_title'].lower()]

#### Creating and testing tools

In [4]:
from src.agents.tools.sql_tool import SQLMovieTool
from src.agents.tools.multimodal_rag import TextMovieTool, VisualMovieTool, CombinedMovieTool
from src.agents.tools.collaborative_filtering import CollaborativeFilteringTool
from src.agents.tools.web_search import WebSearchTool

In [7]:
# Create tools
text_tool = TextMovieTool(full_chain).get_tool()
visual_tool = VisualMovieTool(visual_retriever).get_tool()
combined_tool = CombinedMovieTool(full_chain, visual_retriever).get_tool()
sql_tool = SQLMovieTool(db_path=SQL_PATH).get_tool()

print("=" * 60)
print("Testing TEXT tool:")
print("=" * 60)
result = text_tool.invoke("What is a funny romantic movie set in Europe?")
print(result)

print("\n" + "=" * 60)
print("Testing VISUAL tool:")
print("=" * 60)
result = visual_tool.invoke("dark moody sci-fi films")
print(result)

print("\n" + "=" * 60)
print("Testing COMBINED tool:")
print("=" * 60)
result = combined_tool.invoke("what cartoon is about family and has bright visuals?")
print(result)

print("\n" + "=" * 60)
print("Testing COMBINED tool:")
print("=" * 60)
result = combined_tool.invoke("dark moody sci-fi films")
print(result)

print("\n" + "=" * 60)
print("Testing SQL tool:")
print("=" * 60)
result = sql_tool.invoke("what are the top rated romantic movies made after 2010?")
print(result)

Loaded 6127 critics, 8075 movies, 546252 ratings.
Computed movie similarity matrix.
Testing TEXT tool:


/Users/saghar/Desktop/movie-rag/src/langchain/retrieval/hyde.py:77: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  documents = self.base_retriever.get_relevant_documents(hypothetical_doc)


A funny romantic movie set in Europe is "A Royal Night Out." It features a whimsical depiction of British princesses partying with commoners at the end of World War II, blending romance and comedy in a light-hearted manner. Another option is "Chalet Girl," a feel-good British romcom that offers a witty script and an infectious sense of fun.

Testing VISUAL tool:
The posters listed in the search results represent a variety of films across different genres, but they share a common theme of being darker in tone, which may appeal to audiences looking for moody and intense narratives. 

1. **Dark Blue World** - This film is a drama that likely explores themes of conflict and emotional depth, fitting the dark and moody aesthetic.
2. **Fifty Shades of Grey** - While primarily a romance, it has a darker, more intense atmosphere that aligns with the moody aspect, focusing on complex relationships and emotional struggles.
3. **The King** - A historical drama that likely delves into themes of pow

In [10]:
cf_tool = CollaborativeFilteringTool(REVIEW_PATH).get_tool()

print("\n" + "=" * 60)
print("Testing CF tool:")
print("=" * 60)
result = cf_tool.invoke("star wars, harry potter, kung fu panda 2")
print(result)

Loaded 6127 critics, 8075 movies, 546252 ratings.
Computed movie similarity matrix.

Testing CF tool:
{
  "warnings": "Warning: Multiple close matches found for star wars, it might also not be available in dataset. Options: ['star wars: the clone wars (2008)', 'star wars: episode v - the empire strikes back (1980)', 'rogue one: a star wars story (2016)']. Warning: Multiple close matches found for harry potter, it might also not be available in dataset. Options: ['harry potter and the chamber of secrets (2002)', 'harry potter and the deathly hallows - part 1 (2010)', 'harry potter and the deathly hallows - part 2 (2011)']. ",
  "input_movies": [
    "Kung Fu Panda 2 (2011)"
  ],
  "method": "Item-based collaborative filtering (cosine similarity)",
  "recommendations": [
    {
      "title (year)": "Rango (2011)",
      "similarity_score": 0.528,
      "avg_critic_rating": 7.76,
      "num_critics": 168,
      "reason": "Critics rated this similarly to Kung Fu Panda 2 (2011)"
    },
    

##### I am pretty happy with how we handle movie names, missing movie names, overlapping movie names, and how we respond so that the agent can change their query!!!

In [5]:
web_tool = WebSearchTool(max_results=5).get_tool()

print("\n" + "=" * 60)
print("Testing web search tool:")
print("=" * 60)
result = web_tool.invoke("severence new season 3 release date")
print(result)


Testing web search tool:
Result 1: Severance season 3 is coming: Here's everything we know right now
Realistically, Severance season 3 will probably hit Apple TV in mid-to-late 2027. This 2027 return accounts for about one month of production
Source: https://9to5mac.com/2025/10/20/severance-season-3-heres-everything-we-know-so-far/

Result 2: Severance season 3 release timeline announced - Facebook
Severance returns January 17 Apple TV If season 2 lives up to season 1 I will do an episode by episode analysis Some thoughts made immediately
Source: https://www.facebook.com/groups/389116370609154/posts/761736176680503/

Result 3: 'Severance' Season 3 - News, Cast, Details - Cosmopolitan
We don't have an official premiere date for season 3 of Severance, but by Kier's hand, we won't have to wait three years for it like we did
Source: https://www.cosmopolitan.com/entertainment/tv/a66025894/severance-season-3-news-cast-details/

Result 4: Severance Season 3: Release Date, Cast Changes, and W

#### Testing agent

In [7]:
from src.agents.react import ReactAgent

In [5]:
agent = ReactAgent(full_chain, visual_retriever, SQL_PATH, REVIEW_PATH)

queries = [
    "What is the Matrix about?",
    "Dark moody sci-fi genre films",
    "what themes does titanic touch?",
    "Movies with vibrant colors",
    "drama about love and lust with engaging red poster",
    "top rated romantic movies made after 2012",
    "comedy movies featuring tom hanks with good critic reviews"
]

for query in queries:
    agent.query(query)
    print("\n")

Creating tools...
Building agent graph...
✓ Agent ready!
================================ Human Message =================================

What is the Matrix about?
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_content (call_UGp1CRuZKin1LNw9Ca7tbuIB)
 Call ID: call_UGp1CRuZKin1LNw9Ca7tbuIB
  Args:
    query: What is The Matrix about?


/Users/saghar/Desktop/movie-rag/src/langchain/retrieval/hyde.py:77: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  documents = self.base_retriever.get_relevant_documents(hypothetical_doc)


================================= Tool Message =================================
Name: search_movies_by_content

The provided information does not include a description or details about "The Matrix." Therefore, I cannot answer your question about what "The Matrix" is about.
================================== Ai Message ==================================

"The Matrix" is a science fiction film that explores themes of reality, perception, and control. The story follows a computer hacker named Neo, who discovers that the world he lives in is a simulated reality created by sentient machines to subdue humanity. As he learns the truth about the Matrix, he joins a group of rebels led by Morpheus and Trinity, who fight against the machines to free humanity from this illusion. The film delves into philosophical questions about existence, free will, and the nature of reality, all while featuring groundbreaking visual effects and action sequences.

Done!


================================ Human M

In [11]:
_ = agent.query("recommend some scifi movie with yellow posters from the nineties")

================================ Human Message =================================

recommend some scifi movie with yellow posters from the nineties
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_visual (call_QXKmGeqt0zmTXatOq2WG2qfc)
 Call ID: call_QXKmGeqt0zmTXatOq2WG2qfc
  Args:
    query: sci-fi movies with yellow posters from the nineties
================================= Tool Message =================================
Name: search_movies_by_visual

The posters listed in the search results share a common theme of being visually striking, with a focus on the science fiction genre, particularly in the context of the nineties. The films include a mix of action, adventure, horror, and drama, indicating a diverse range of storytelling within the sci-fi realm. 

1. **Virus (1999)** - This film combines elements of action, adventure, and horror, suggesting a thrilling narrative involving technology or extraterrestrial threats.

In [12]:
_ = agent.query("what's that movie where the kid is left alone")

================================ Human Message =================================

what's that movie where the kid is left alone
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_content (call_gA5sadQAStQU6pRnR2RjmKNX)
 Call ID: call_gA5sadQAStQU6pRnR2RjmKNX
  Args:
    query: kid left alone
================================= Tool Message =================================
Name: search_movies_by_content

The movie "The Ant Bully" features a young boy named Lucas who is navigating his independence in a big world, which may resonate with the theme of a kid left alone. Additionally, "Walkabout" involves children who are left to survive in the wilderness, highlighting their struggle and need to return to civilization. Both films explore themes related to children facing challenges on their own.
================================== Ai Message ==================================

The movie you're thinking of might be "Home Alone," where

In [8]:
_ = agent.query("what movie should I watch tonight if I really enjoyed watching star wars and harry potter? kung fu panda was kinda fun too")

================================ Human Message =================================

what movie should I watch tonight if I really enjoyed watching star wars and harry potter? kung fu panda was kinda fun too
================================== Ai Message ==================================
Tool Calls:
  recommend_by_similar_taste (call_5KdsqzS0UU4qkXnijGtVZtZX)
 Call ID: call_5KdsqzS0UU4qkXnijGtVZtZX
  Args:
    movie_titles: Star Wars, Harry Potter, Kung Fu Panda
  search_movies_by_content (call_YeUehXcx87SPjqChhLyvdKIM)
 Call ID: call_YeUehXcx87SPjqChhLyvdKIM
  Args:
    query: fantasy adventure with elements of magic and action
================================= Tool Message =================================
Name: search_movies_by_content

"Warcraft" (2016) is a fantasy adventure film that features elements of magic and action. Directed by Duncan Jones, it explores the conflict between the Orcs and the inhabitants of Azeroth, involving powerful warlocks, magical portals, and epic battles.

In [7]:
_ = agent.query("movies similar to star wars and harry potter and kung fu panda based on critic ratings")

================================ Human Message =================================

movies similar to star wars and harry potter and kung fu panda based on critic ratings
================================== Ai Message ==================================
Tool Calls:
  recommend_by_similar_taste (call_eElmovdLfV6fNg1GgGCPXq1J)
 Call ID: call_eElmovdLfV6fNg1GgGCPXq1J
  Args:
    movie_titles: Star Wars, Harry Potter, Kung Fu Panda
================================= Tool Message =================================
Name: recommend_by_similar_taste

{"warnings": "Warning: Multiple close matches found for Star Wars, it might also not be available in dataset. Options: ['star wars: the clone wars (2008)', 'star wars: episode v - the empire strikes back (1980)', 'rogue one: a star wars story (2016)']. Warning: Multiple close matches found for Harry Potter, it might also not be available in dataset. Options: ['harry potter and the chamber of secrets (2002)', 'harry potter and the deathly hallows - part 

In [8]:
agent = ReactAgent(full_chain, visual_retriever, SQL_PATH, REVIEW_PATH, web_search_enabled=True)

_ = agent.query("What's the highest grossing movie of 2025?")

Creating tools...
Loaded 6127 critics, 8075 movies, 546252 ratings.
Computed movie similarity matrix.
Building agent graph...
✓ Agent ready!
================================ Human Message =================================

What's the highest grossing movie of 2025?
================================== Ai Message ==================================
Tool Calls:
  search_web_for_movies (call_6Hw7xSqu3MZPlFkqKfoG7Ph5)
 Call ID: call_6Hw7xSqu3MZPlFkqKfoG7Ph5
  Args:
    query: highest grossing movie of 2025
================================= Tool Message =================================
Name: search_web_for_movies

Result 1: Highest-Grossing Movies Of 2025: The 10 Films That Made The ...
Image 3: Highest-Grossing Movies Of 2025: The 10 Films That Made The Most Money Rounding out the highest grossing movies of 2025, _Captain America: Brave New World_ brought in over $415 million (Box Office Mojo ) worldwide. The highest grossing movie of 2025 is _Ne Zha 2_, which has earned more than $1.9 billi

#### Just to see what query the sql tool generates

In [13]:
# short term SQLMovieTool was made that prints out query. Don't run this again
sql_tool = SQLMovieTool(db_path=SQL_PATH).get_tool()

queries = [
    "top rated romantic movies made after 2012",
    "comedy movies featuring tom hanks with good critic reviews"
]

for query in queries:
    sql_tool(query)
    print("\n")

SELECT movie_title, release_year, imdb_rating, tomatometer_rating, audience_rating
FROM movies
WHERE genres LIKE '%Romance%'
AND release_year > 2012
ORDER BY imdb_rating DESC, tomatometer_rating DESC, audience_rating DESC;


SELECT * 
FROM movies 
WHERE genres LIKE '%Comedy%' 
AND actors LIKE '%Tom Hanks%' 
AND tomatometer_rating >= 75;


